# Dydra API client

> Tools to interact with the Dydra API

In [ ]:
#| default_exp api

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import requests
import rdflib
from dotenv import load_dotenv
import os
from SPARQLWrapper import SPARQLWrapper, JSON

In [ ]:
#| export
class DydraClient():    
    """
    A client for interacting with a Dydra SPARQL endpoint.

    Attributes:
        endpoint (str): The SPARQL endpoint URL.
        api_key (str): The API key for authorization.
    """

    @staticmethod
    def load_env(path):
        """
        Loads environment variables from a specified .env file.
        
        Args:
            path (str): Path to the .env file.
            
        Returns:
            tuple: Contains the endpoint and API key as strings.
        """

        load_dotenv(path)

        return os.getenv("DYDRA_ENDPOINT"), os.getenv("DYDRA_API_KEY")


    def __init__(self, endpoint, api_key):
        """
        Initializes the DydraClient with the given endpoint and API key.
        
        Args:
            endpoint (str): The SPARQL endpoint URL.
            api_key (str): The API key for authorization.
        """
        self.endpoint = endpoint
        self.api_key = api_key

    def import_by_file(self, file_path, format, graph_uri=None):
        """
        Imports RDF data from a file into the Dydra store.

        Args:
            file_path (str): The path to the RDF file to import.
            format (str): The format of the RDF file (e.g., 'xml', 'nt').
            graph_uri (str, optional): URI of the graph where data will be inserted. Defaults to None.
        """

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/sparql-update"
        }

        # RDFファイルの読み込み
        graph = rdflib.Graph()
        graph.parse(file_path, format=format)  # フォーマットはファイルに応じて変更

        nt_data = graph.serialize(format='nt')

        if graph_uri is None:
            query = f"""
            INSERT DATA {{
            {nt_data}
            }}
            """

        else:
            query = f"""
            INSERT DATA {{
            GRAPH <{graph_uri}> {{
                {nt_data}
            }}
            }}
            """

        

        response = requests.post(self.endpoint, data=query, headers=headers)
        if response.status_code == 200:
            print("Data successfully inserted.")
        else:
            print(f"Error: {response.status_code} {response.text}")

    def query(self, query):
        """
        Executes a SPARQL query against the endpoint and returns the results.

        Args:
            query (str): The SPARQL query to execute.

        Returns:
            dict: The query results in JSON format.
        """

        # SPARQLエンドポイントの設定
        sparql = SPARQLWrapper(self.endpoint)

        # クエリの設定
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)

        results = sparql.query().convert()

        return results
    
    def clear(self, graph_uri=None):
        """
        Clears all data from the default or specified graph.

        Args:
            graph_uri (str, optional): URI of the graph to clear. Defaults to None.
        """

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/sparql-update"
        }

        if graph_uri is None:
            query = """
            DELETE {
            ?s ?p ?o
            }
            WHERE {
            ?s ?p ?o
            }
            """

        else:
            query = f"""
            DELETE {{
            GRAPH <{graph_uri}> {{
                ?s ?p ?o
            }}
            }}
            WHERE {{
            GRAPH <{graph_uri}> {{
                ?s ?p ?o
            }}
            }}
            """

        response = requests.post(self.endpoint, data=query, headers=headers)

        if response.status_code == 200:
            print("Data successfully cleared.")
        else:
            print(f"Error: {response.status_code} {response.text}")


In [ ]:
show_doc(DydraClient.import_by_file)

---

[source](https://github.com/nakamura196/dydra-py/blob/main/dydra_py/core.py#L29){target="_blank" style="float:right; font-size:smaller"}

### DydraClient.import_by_file

>      DydraClient.import_by_file (file_path, format, graph_uri=None)

*Imports RDF data from a file into the Dydra store.

Args:
    file_path (str): The path to the RDF file to import.
    format (str): The format of the RDF file (e.g., 'xml', 'nt').
    graph_uri (str, optional): URI of the graph where data will be inserted. Defaults to None.*

In [ ]:
show_doc(DydraClient.query)

---

[source](https://github.com/nakamura196/dydra-py/blob/main/dydra_py/core.py#L66){target="_blank" style="float:right; font-size:smaller"}

### DydraClient.query

>      DydraClient.query (query)

*Executes a SPARQL query against the endpoint and returns the results.

Args:
    query (str): The SPARQL query to execute.

Returns:
    dict: The query results in JSON format.*

In [ ]:
show_doc(DydraClient.clear)

---

[source](https://github.com/nakamura196/dydra-py/blob/main/dydra_py/core.py#L82){target="_blank" style="float:right; font-size:smaller"}

### DydraClient.clear

>      DydraClient.clear (graph_uri=None)

*Clears all data from the default or specified graph.

Args:
    graph_uri (str, optional): URI of the graph to clear. Defaults to None.*

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()